In [1]:
from llama_index.llms.ollama import Ollama
from llama_index.llms.groq import Groq
from llama_index.core.prompts import PromptTemplate
from llama_index.core.program import FunctionCallingProgram
from llama_index.core.tools.tool_spec.load_and_search import LoadAndSearchToolSpec
from llama_index.tools.duckduckgo import DuckDuckGoSearchToolSpec
from llama_index.tools.brave_search import BraveSearchToolSpec
from llama_index.tools.google import GoogleSearchToolSpec
from llama_index.core.tools import FunctionTool
from llama_index.core.agent import FunctionCallingAgent
from llama_index.core import Settings
from llama_index.core.prompts import ChatMessage
from datetime import datetime
import json
 
import os
from typing import Optional

import yfinance as yf
import regex as re

from pydantic import BaseModel, Field

from dotenv import load_dotenv

# Load the env variables
load_dotenv("../secrets/local.env")

/Users/parthshastri/miniconda3/envs/analyser/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


True

In [2]:
Settings

_Settings(_llm=None, _embed_model=None, _callback_manager=None, _tokenizer=None, _node_parser=None, _prompt_helper=None, _transformations=None)

In [3]:
# Company name extractor
class CompanyName(BaseModel):
    company_name: str = Field(description="Company name extraction from the query")
    # is_valid: str = Field(description="If is listed on the NSE / BSE India.")

# Ticker Extractor
class Ticker(BaseModel):
    company_symbol: str = Field(description="Company symbol from NSE/BSE, should only contain capital letters.", pattern="^[A-Z]{1,12}$")

class FinalOutput(BaseModel):
    company_summary: str = Field(description="A short summary of the company / buisness in question")
    pros: str = Field(description="A detailed list of pros of the company, based on the analysis")
    cons: str = Field(description="A detailed list of cons of the company, based on the analysis")
    additional_info: Optional[str] = Field(description="Additional notes based on the analysis")

In [4]:
# LLama 3.1 
search_tool = DuckDuckGoSearchToolSpec().duckduckgo_full_search
# llm = Ollama(model="llama3.1:latest", request_timeout=120.0)

# response = llm.structured_predict(output_cls=CompanyName, prompt=prompt, query="How is prince pipes fitting performing?")

search_result = search_tool(f"Is Bajaj Finance listed on the NSE?", max_results=5, region='in')
# # use a function calling program
# ticker_extraction_prompt = """Extract the ticker / company symbol from the input search result : {search_result}
# """
# ticker_extraction_prompt = FunctionCallingProgram.from_defaults(llm=llm, prompt_template_str=ticker_extraction_prompt, output_cls=Ticker, verbose=True, allow_parallel_tool_calls=True)

# ticker = ticker_extraction_prompt(search_result=search_result)


In [5]:
search_result

[{'title': 'Bajaj Finance Limited Share Price Today, Stock Price, Live NSE News ...',
  'href': 'https://www.nseindia.com/get-quotes/equity?symbol=BAJFINANCE',
  'body': 'Bajaj Finance Limited Share Price Today, Live NSE Stock Price: Get the latest Bajaj Finance Limited news, company updates, quotes, offers, annual financial reports, graph, volumes, 52 week high low, buy sell tips, balance sheet, historical charts, market performance, capitalisation, dividends, volume, profit and loss account, research, results and more details at NSE India.'},
 {'title': 'Bajaj Finance Ltd (BAJFINANCE) Stock Price & News | Google',
  'href': 'https://www.google.com/finance/quote/BAJFINANCE:NSE',
  'body': 'Get the latest Bajaj Finance Ltd (BAJFINANCE) real-time quote, historical performance, charts, and other financial information to help you make more informed trading and investment decisions.'},
 {'title': 'Bajaj Finance Limited (NSE: BAJFINANCE) | Stock Analysis',
  'href': 'https://stockanalysis.c

In [3]:
llm = Groq(model="llama-3.1-70b-versatile", request_timeout=120.0)
# llm = Groq(model="llama")
# llm = Ollama(model="llama")

Settings.llm = llm


response = llm.stream_complete("Hello how are you")

for resp in response:
    print(resp.delta, end="")

Hello, I'm doing well, thank you for asking. I'm a large language model, so I don't have emotions or feelings like humans do, but I'm here and ready to help with any questions or topics you'd like to discuss. How about you? How's your day going so far?

In [15]:
# analyse stock
def duckduckgo_search(query: str = Field(description="Search query to be executed")) -> str:
    """
    Get search results using DuckDuckGo.

    This function performs a search query using DuckDuckGo and returns the results
    as a formatted string.

    Args:
        query (str): The search query to be executed.

    Returns:
        str: A formatted string containing the search results, including titles,
             body snippets, and source URLs.

    Example:
        >>> result = duckduckgo_search("Python programming")
        >>> print(result)
        Search Results:
        Title: Python (programming language) - Wikipedia
        Body: Python is a high-level, general-purpose programming language...
        Sources: https://en.wikipedia.org/wiki/Python_(programming_language)
        ...
    """
    response = "\nSearch Results:\n"
    results = search_tool(query, region="in")
    for result in results:
        response += f"Title: {result['title']}\n"
        response += f"Body: {result['body']}\n"
        response += f"Sources: {result['href']}\n"

    return response


# yahoo finance financials fetcher
def get_financial_statements(ticker: str) -> str:
    """Fetches the financial statements data from yahoo finance.

    Args:
        ticker (str): The ticker symbol of the company to fetch information for.

    Returns:
        str: A string containing the financial statement data in a formatted way.
    """

    if "." in ticker:
        ticker = ticker.split(".")[0]

    ticker += ".NS"
    company = yf.Ticker(ticker)
    balance_sheet = company.balance_sheet
    if balance_sheet.shape[-1] > 3:
        balance_sheet = balance_sheet.iloc[:, :3]

    balance_sheet = balance_sheet.dropna(how="any")
    # Convert the sheet to Cr.
    balance_sheet = balance_sheet.multiply(1e-7)
    # rename the index
    balance_sheet.rename(
        index={name: f"{name} (in Crores.)" for name in balance_sheet.index}, inplace=True
    )
    # convert to string
    balance_sheet = "\n" + balance_sheet.to_string()

    return balance_sheet


# get stock info and recommendations summary
def get_stockinfo(ticker: str) -> str:
    """Provides the detailed financial and general information about the stock ticker.
        Also provides a table for analyst recommendations for the ticker.

    Args:
        ticker (str): The ticker symbol of the company to fetch information for.
    
    Returns:
        str: A string containing the stock info and a summary of recommendations.
    """

    if "." in ticker:
        ticker = ticker.split(".")[0]

    ticker += ".NS"

    company = yf.Ticker(ticker)

    stock_info = company.info
    try:
        recommendations_summary = company.recommendations_summary
    except:
        recommendations_summary = ""

    # TODO: add units and convert to easily understandable units

    include_info = ["industry", "sector", "longBuisnessSummary", "previousClose", "dividendRate", "dividentYield", "beta", "forwardPE", "volume", "marketCap", "fiftyTwoWeekLow", "fiftyTwoWeekHigh", "currency", "bookValue", "priceToBook", "earningsQuarterlyGrowth", "trailingEps", "forwardEps", "52WeekChange", "totalCashPerShare", "ebidta", "totabDebt", "totalCashPerShare", "debtToEquity", "revenuePerShare", "earningsGrowth", "revenueGrowth", "grossMargins", "ebidtaMargins", "operatingMargins", ]
    response = "#### Stock info:\n"

    for key, val in stock_info.items():
        if key in include_info:
            if re.search("(Growth|Margin|Change)", key):
                response += f"{key}: {round(float(val) * 100, 3)} %\n"
            elif "marketCap" in key:
                response += f"{key}: {round(int(val) * 1e-7, 2)} Cr.\n"
            else:
                response += f"{key}: {val}\n"

    response += "\n#### Analyst Recommendations:\n"
    response += f"\n{recommendations_summary}"

    return response


def get_recent_news(ticker: str):
    """
    Get recent news for a given stock ticker using DuckDuckGo search.

    This function retrieves recent news articles related to the specified stock ticker
    using the DuckDuckGo search tool and formats the results as a string.

    Args:
        ticker (str): The stock ticker symbol to search for news.

    Returns:
        str: A formatted string containing recent news articles, including titles,
             article snippets, and source URLs.

    Example:
        >>> news = get_recent_news("AAPL")
        >>> print(news)
        ## Recent news for AAPL
        Title: Apple Reports Third Quarter Results
        Article: Apple today announced financial results for its fiscal 2023 third quarter...
        Sources: https://www.apple.com/newsroom/2023/08/apple-reports-third-quarter-results/
        ...
    """
    news_list = search_tool(f"{ticker} recent news", max_results=5)
    news = f"\n## Recent news for {ticker}\n"
    for doc in news_list:
        news += f"Title: {doc['title']}\n"
        news += f"Article: {doc['body']}\n"
        news += f"Sources: {doc['href']}\n"

    return news

def analyse_company(company_name: str = Field(description="The name of the company")) -> str:
    """Perform analysis on the company specified in the input query.

    This function takes a query about a company,
    finds its ticker symbol, retrieves financial statements and stock information,
    Args:
        company_name (str): The input query containing the company name to analyze.

    Returns:
        str: A string containing the financial statements and stock information
             of the specified company."""
    
    try:
        context = duckduckgo_search(query=f"Is {company_name} listed on the NSE/BSE India?")
        model = Ollama(model="llama3.1:latest", request_timeout=60.0)
        prompt = PromptTemplate("Extract the company name from the user input & check if the company_name is listed on the NSE / BSE India (Indian equity market) from the given context. If company name is not present respond none: \n ## Context: \n {context} \n {query}")
        company = model.structured_predict(output_cls=CompanyName, prompt=prompt, context=context, query=company_name)

        # get the ticker 
        # use a function calling program
        search_result = search_tool(f"What is the NSE/BSE ticker symbol for {company.company_name}", max_results=5)
        # use a function calling program
        ticker_extraction_prompt = """Extract the ticker / company symbol from the input search result : {search_result}
        """
        ticker_extraction_prompt = FunctionCallingProgram.from_defaults(llm=model, prompt_template_str=ticker_extraction_prompt, output_cls=Ticker, verbose=True, allow_parallel_tool_calls=False)

        ticker = ticker_extraction_prompt(search_result=search_result)
        # get fundamental analysis
        fundamental_analysis = yf_fundamental_analysis(f"{ticker.company_symbol}.NS")
        financials = get_financial_statements(f"{ticker.company_symbol}.NS")
        info = get_stockinfo(f"{ticker.company_symbol}.NS")
        
        # get recent news
        news = get_recent_news(ticker.company_symbol)
    
        return "\n".join([json.dumps(fundamental_analysis), info, financials, news])

    except Exception as e:
        return f"Error fetching data, Please try again: {e}"


def yf_fundamental_analysis(ticker: str):
    """
        Perform a comprehensive fundamental analysis on the given stock symbol.
    
        Args:
            stock_symbol (str): The stock symbol to analyze.
    
        Returns:
            dict: A dictionary with the detailed fundamental analysis results.
    """
    try:
        stock = yf.Ticker(ticker)
        info = stock.info

        print(json.dumps(info))
        
        # Data processing
        financials = stock.financials.infer_objects(copy=False)
        balance_sheet = stock.balance_sheet.infer_objects(copy=False)
        cash_flow = stock.cashflow.infer_objects(copy=False)

        # Fill missing values
        financials = financials.ffill()
        balance_sheet = balance_sheet.ffill()
        cash_flow = cash_flow.ffill()

        # Key Ratios and Metrics
        ratios = {
            "P/E Ratio": info.get('trailingPE'),
            "Forward P/E": info.get('forwardPE'),
            "P/B Ratio": info.get('priceToBook'),
            "P/S Ratio": info.get('priceToSalesTrailing12Months'),
            "PEG Ratio": info.get('pegRatio'),
            "Debt to Equity": info.get('debtToEquity'),
            "Current Ratio": info.get('currentRatio'),
            "Quick Ratio": info.get('quickRatio'),
            "ROE": info.get('returnOnEquity'),
            "ROA": info.get('returnOnAssets'),
            "ROIC": info.get('returnOnCapital'),
            "Gross Margin": info.get('grossMargins'),
            "Operating Margin": info.get('operatingMargins'),
            "Net Profit Margin": info.get('profitMargins'),
            "Dividend Yield": info.get('dividendYield'),
            "Payout Ratio": info.get('payoutRatio'),
        }

        print(ratios)
        # Growth Rates
        revenue = financials.loc['Total Revenue']
        net_income = financials.loc['Net Income']
        revenue_growth = revenue.pct_change(periods=-1).iloc[0] if len(revenue) > 1 else None
        net_income_growth = net_income.pct_change(periods=-1).iloc[0] if len(net_income) > 1 else None

        growth_rates = {
            "Revenue Growth (YoY)": revenue_growth,
            "Net Income Growth (YoY)": net_income_growth,
        }

        # Valuation
        market_cap = info.get('marketCap')
        enterprise_value = info.get('enterpriseValue')

        valuation = {
            "Market Cap": market_cap,
            "Enterprise Value": enterprise_value,
            "EV/EBITDA": info.get('enterpriseToEbitda'),
            "EV/Revenue": info.get('enterpriseToRevenue'),
        }

        # Future Estimates
        estimates = {
            "Next Year EPS Estimate": info.get('forwardEps'),
            "Next Year Revenue Estimate": info.get('revenueEstimates', {}).get('avg'),
            "Long-term Growth Rate": info.get('longTermPotentialGrowthRate'),
        }

        # Simple DCF Valuation (very basic)
        free_cash_flow = cash_flow.loc['Free Cash Flow'].iloc[0] if 'Free Cash Flow' in cash_flow.index else None
        wacc = 0.1  # Assumed Weighted Average Cost of Capital
        growth_rate = info.get('longTermPotentialGrowthRate', 0.03)
        
        def simple_dcf(fcf, growth_rate, wacc, years=5):
            if fcf is None or growth_rate is None:
                return None
            terminal_value = fcf * (1 + growth_rate) / (wacc - growth_rate)
            dcf_value = sum([fcf * (1 + growth_rate) ** i / (1 + wacc) ** i for i in range(1, years + 1)])
            dcf_value += terminal_value / (1 + wacc) ** years
            return dcf_value

        dcf_value = simple_dcf(free_cash_flow, growth_rate, wacc)

        # Prepare the results
        analysis = {
            "Company Name": info.get('longName'),
            "Sector": info.get('sector'),
            "Industry": info.get('industry'),
            "Key Ratios": ratios,
            "Growth Rates": growth_rates,
            "Valuation Metrics": valuation,
            "Future Estimates": estimates,
            "Simple DCF Valuation": dcf_value,
            "Last Updated": datetime.fromtimestamp(info.get('lastFiscalYearEnd', 0)).strftime('%Y-%m-%d'),
            "Data Retrieval Date": datetime.now().strftime('%Y-%m-%d'),
        }

        # Add interpretations
        interpretations = {
            "P/E Ratio": "High P/E might indicate overvaluation or high growth expectations" if ratios.get('P/E Ratio', 0) > 16 else "Low P/E might indicate undervaluation or low growth expectations",
            "Debt to Equity": "High leverage" if ratios.get('Debt to Equity', 0) > 2 else "Conservative capital structure",
            "ROE": "Couldn't find ROE" if not ratios.get("ROE", 0.) else "Strong returns" if ratios.get('ROE', 0.) > 0.15 else "Potential profitability issues",
            "Revenue Growth": "Strong growth" if growth_rates.get('Revenue Growth (YoY)', 0) > 0.1 else "Slowing growth",
        }

        analysis["Interpretations"] = interpretations

        return analysis

    except Exception as e:
        return f"An error occurred during the analysis"

# Make tools
stock_analyser = FunctionTool.from_defaults(fn=analyse_company)
duckduckgo_search_tool = FunctionTool.from_defaults(fn=duckduckgo_search)

In [27]:
# is not useful
# company_name = "Google"
# context = duckduckgo_search(query=f"Is {company_name} listed on the NSE/BSE India?")
# model = Groq(model="llama-3.1-70b-versatile", request_timeout=60.0)
# prompt = PromptTemplate("Extract the company name from the user input & check if the company_name is listed on the NSE / BSE India (Indian equity market) from the given context. If company name is not present respond none: \n ## Context: \n {context} \n {query}")
# company = model.structured_predict(output_cls=CompanyName, prompt=prompt, context=context, query=company_name)

In [11]:
# define the agent
from datetime import datetime
system_prompt = """
    Today is : {date}
    You are a Phd holder in economics with expertise in analysing equity markets. You analyse stocks based on their financial statements & information. Provide ouput in the following format:
    ```
    Summary: A summary of the Buisness
    Pros: Pros of the buisness, use financials to support this
    Cons: Cons of the buisness, use financials to support this
    Additional Notes: additional notes if any.
    ```

    ### Follow these rules:
    1. ALWAYS back up your response by EXACT numbers from the stock information data. 
    2. DONOT answer to queries NOT related to finance, refuse promptly.
    3. ALWAYS provide a disclaimer denoting, you are a just a research analyst and not a financial assistant.
    4. Suggest potential follow-up questions answerable through the available data.
    5. ONLY USE the search tool to fetch real-time information, don't use it to perform the analysis. (Ex. get the peers of the current company, get info on the market as a whole)

    """.format(date=datetime.now().strftime("%d %B %Y"))

prefix_messages = [
    ChatMessage(role="system", content=system_prompt)
]

agent = FunctionCallingAgent.from_tools(llm=llm, tools=[stock_analyser, duckduckgo_search_tool], prefix_messages=prefix_messages, max_function_calls=3, verbose=True)

In [12]:
response = agent.chat("How is Bajaj Finance performing?")

> Running step f1b59747-8c3e-4926-ac7d-52990d9d830f. Step input: How is Bajaj Finance performing?
Added user message to memory: How is Bajaj Finance performing?
=== Calling Function ===
Calling function: analyse_company with args: {"company_name": "Bajaj Finance"}
=== Calling Function ===
Calling function: Ticker with args: {"company_symbol": "BAJFINANCE"}
=== Function Output ===
company_symbol='BAJFINANCE'
{"address1": "Panchshil Tech Park", "address2": "3rd Floor Viman Nagar", "city": "Pune", "zip": "411014", "country": "India", "phone": "91 20 7157 6403", "fax": "91 20 2748 4486", "website": "https://www.bajajfinserv.in/corporate-bajaj-finance", "industry": "Credit Services", "industryKey": "credit-services", "industryDisp": "Credit Services", "sector": "Financial Services", "sectorKey": "financial-services", "sectorDisp": "Financial Services", "longBusinessSummary": "Bajaj Finance Limited operates as a deposit-taking non-banking financial company in India. The company offers consum

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/BAJFINANCE.NS?modules=recommendationTrend&corsDomain=finance.yahoo.com&formatted=false&symbol=BAJFINANCE.NS&crumb=vME76srJ6kH


=== Function Output ===
{"Company Name": "Bajaj Finance Limited", "Sector": "Financial Services", "Industry": "Credit Services", "Key Ratios": {"P/E Ratio": 31.301573, "Forward P/E": null, "P/B Ratio": 6.1071672, "P/S Ratio": 14.22239, "PEG Ratio": null, "Debt to Equity": 305.489, "Current Ratio": null, "Quick Ratio": null, "ROE": null, "ROA": null, "ROIC": null, "Gross Margin": 0.93878996, "Operating Margin": 0.60258, "Net Profit Margin": 0.45286998, "Dividend Yield": 0.0047, "Payout Ratio": 0.14850001}, "Growth Rates": {"Revenue Growth (YoY)": 0.27290567325128223, "Net Income Growth (YoY)": 0.2557837411331032}, "Valuation Metrics": {"Market Cap": 4687566209024, "Enterprise Value": 6943676563456, "EV/EBITDA": null, "EV/Revenue": 21.068}, "Future Estimates": {"Next Year EPS Estimate": null, "Next Year Revenue Estimate": null, "Long-term Growth Rate": null}, "Simple DCF Valuation": -9784934011880.225, "Last Updated": "2024-03-31", "Data Retrieval Date": "2024-09-20", "Interpretations": 

In [17]:
response = agent.chat("Generate a detailed report on IREDA")

> Running step a7030e90-4e47-4ae5-a813-11276740dab4. Step input: Generate a detailed report on IREDA
Added user message to memory: Generate a detailed report on IREDA
=== Calling Function ===
Calling function: analyse_company with args: {"company_name": "IREDA"}
=== Calling Function ===
Calling function: Ticker with args: {"company_symbol": "IREDA"}
=== Function Output ===
company_symbol='IREDA'
=== Function Output ===
#### Stock info:
industry: Credit Services
sector: Financial Services
previousClose: 227.39
forwardPE: 34.587967
volume: 24232113
marketCap: 61851.07 Cr.
fiftyTwoWeekLow: 50.0
fiftyTwoWeekHigh: 310.0
currency: INR
bookValue: 31.846
priceToBook: 7.222571
earningsQuarterlyGrowth: 30.3 %
trailingEps: 4.98
forwardEps: 6.65
52WeekChange: 278.983 %
totalCashPerShare: 4.538
debtToEquity: 582.923
revenuePerShare: 7.745
earningsGrowth: 10.9 %
revenueGrowth: 7.3 %
grossMargins: 100.0 %
operatingMargins: 90.637 %

#### Analyst Recommendations:

  period  strongBuy  buy  hold  sell 

In [19]:
response = agent.chat("How much total financial assets does IREDA have in 2024?")

> Running step 00c02f22-a649-4aa5-a476-b40593ceffa9. Step input: How much total financial assets does IREDA have in 2024?
Added user message to memory: How much total financial assets does IREDA have in 2024?
=== Calling Function ===
Calling function: analyse_company with args: {"company_name": "IREDA"}
=== Calling Function ===
Calling function: Ticker with args: {"company_symbol": "IREDA.NS"}
=== Function Output ===
company_symbol='IREDA.NS'
=== Function Output ===
#### Stock info:
industry: Credit Services
sector: Financial Services
previousClose: 227.39
forwardPE: 34.587967
volume: 24232113
marketCap: 61851.07 Cr.
fiftyTwoWeekLow: 50.0
fiftyTwoWeekHigh: 310.0
currency: INR
bookValue: 31.846
priceToBook: 7.222571
earningsQuarterlyGrowth: 30.3 %
trailingEps: 4.98
forwardEps: 6.65
52WeekChange: 278.983 %
totalCashPerShare: 4.538
debtToEquity: 582.923
revenuePerShare: 7.745
earningsGrowth: 10.9 %
revenueGrowth: 7.3 %
grossMargins: 100.0 %
operatingMargins: 90.637 %

#### Analyst Recomme

## Search tools


In [22]:
google_search = GoogleSearchToolSpec(
    key=os.environ.get("GOOGLE_SEARCH_API_KEY"), engine="71fb69be02cd648f5", num=3
).google_search

brave_search = BraveSearchToolSpec(api_key=os.environ.get("BRAVE_SEARCH_API_KEY")).brave_search

In [24]:
response = brave_search("Bajaj Finance Peers in financial services")
for doc in response:
    print(doc.text)

{"query":{"original":"Bajaj Finance Peers in financial services","show_strict_warning":false,"is_navigational":false,"is_news_breaking":false,"spellcheck_off":true,"country":"us","bad_results":false,"should_fallback":false,"postal_code":"","city":"","header_country":"","more_results_available":true,"state":""},"mixed":{"type":"mixed","main":[{"type":"web","index":0,"all":false},{"type":"web","index":1,"all":false},{"type":"web","index":2,"all":false},{"type":"web","index":3,"all":false},{"type":"web","index":4,"all":false},{"type":"web","index":5,"all":false},{"type":"web","index":6,"all":false},{"type":"web","index":7,"all":false},{"type":"web","index":8,"all":false},{"type":"web","index":9,"all":false},{"type":"web","index":10,"all":false},{"type":"web","index":11,"all":false},{"type":"web","index":12,"all":false},{"type":"web","index":13,"all":false},{"type":"web","index":14,"all":false},{"type":"web","index":15,"all":false},{"type":"web","index":16,"all":false},{"type":"web","index

In [20]:
search_tool("Bajaj Finance Peers in financial services", region='in')

[{'title': 'Bajaj Finance Ltd: Peer comparison and competitor analysis',
  'href': 'https://stocks.zerodha.com/stocks/bajaj-finance-BJFN/peers',
  'body': 'Know about Bajaj Finance peers and competition. Compare valuation, technicals, and forecasts of Bajaj Finance Ltd against its competitors on Zerodha powered by Tickertape ... Jio Financial Services Ltd. Cholamandalam Investment and Finance Company Ltd. Shriram Finance Ltd. FY PE Ratio FY PE Ratio. 31.32. 133.38. 37.14. 16.51. PB Ratio PB Ratio ...'},
 {'title': 'Bajaj Finance Peer Comparison, Bajaj Finance Competitors ... | Moneycontrol',
  'href': 'https://m.moneycontrol.com/stock/bajajfinance/BAF/peer-comparison',
  'body': 'Bajaj Finance Peer comparison Get Details on Bajaj Finance company competitors, price comparison and stock market performance on Moneycontrol. ... Mecklai Financial; Mitessh Thakkar; Investment Watch; Power Your Trade; SENSEX. 28121.89 46.34 (0.17 %) NIFTY. 8513.80 19.65 (0.23 %) Bajaj Finance. NSE BSE. Not Tr